[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ApzOccU5rfx4zvh_irK8k82cxP_EurjH?usp=sharing)

# Generative summarization with enconder-decoder model

reference: https://keras.io/examples/nlp/abstractive_summarization_with_bart/


## Introduction

[Bidirectional Autoregressive Transformer (BART)](https://arxiv.org/abs/1910.13461)
is a Transformer-based encoder-decoder model, often used for
sequence-to-sequence tasks like summarization and neural machine translation.
BART is pre-trained in a self-supervised fashion on a large text corpus.

During
pre-training, the text is corrupted and BART is trained to reconstruct the
original text (hence called a "denoising autoencoder"). Some pre-training tasks
include token masking, token deletion, sentence permutation (shuffle sentences
and train BART to fix the order), etc.

Review all potential architectures that you can use in Keras:
- https://keras.io/keras_hub/api/models/

Check more model checkpoints in the hub:
- https://www.kaggle.com/models

## Setup

Before we start implementing the pipeline, let's install and import all the
libraries we need. We'll be using the KerasHub library. We will also need a
couple of utility libraries.

- **py7zr** is a library and utility to support 7zip archive compression, decompression, encryption and decryption written by Python programming language.
- **keras-hub**: download pretrained models from hub (https://www.kaggle.com/models/)

In [ ]:
!pip install git+https://github.com/keras-team/keras-hub.git py7zr -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 25.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.21.1 requires keras-hub==0.21.1, but you have keras-hub 0.23.0.dev0 which is incompatible.


Import all necessary libraries.

In [ ]:
import py7zr
import time

import keras_hub
import keras
import tensorflow as tf
import tensorflow_datasets as tfds

Let's also define our hyperparameters.

In [ ]:
BATCH_SIZE = 8
NUM_BATCHES = 600
EPOCHS = 1
MAX_ENCODER_SEQUENCE_LENGTH = 512
MAX_DECODER_SEQUENCE_LENGTH = 128
MAX_GENERATION_LENGTH = 40

## Dataset

Let's load the [SAMSum dataset](https://arxiv.org/abs/1911.12237). This dataset
contains around 15,000 pairs of conversations/dialogues and summaries.

In [ ]:
# Download the dataset manually (only once)
filename = keras.utils.get_file(
    "corpus.7z",
    origin="https://arxiv.org/src/1911.12237v2/anc/corpus.7z",
)

# Extract it
with py7zr.SevenZipFile(filename, mode="r") as z:
    z.extractall(path="/root/tensorflow_datasets/downloads/manual")

# Now load via TFDS
samsum_ds = tfds.load("samsum", split="train", as_supervised=True)

2944100/2944100 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/samsum/incomplete.6NRBCL_1.0.0/samsum-train.tfrecord*...:   0%|          |…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/samsum/incomplete.6NRBCL_1.0.0/samsum-validation.tfrecord*...:   0%|      …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/samsum/incomplete.6NRBCL_1.0.0/samsum-test.tfrecord*...:   0%|          | …

Dataset samsum downloaded and prepared to /root/tensorflow_datasets/samsum/1.0.0. Subsequent calls will reuse this data.


The dataset has two fields: `dialogue` and `summary`. Let's see a sample.

In [ ]:
for dialogue, summary in samsum_ds:
    print(dialogue.numpy())
    print(summary.numpy())
    break

b"Carter: Hey Alexis, I just wanted to let you know that I had a really nice time with you tonight. \r\nAlexis: Thanks Carter. Yeah, I really enjoyed myself as well. \r\nCarter: If you are up for it, I would really like to see you again soon.\r\nAlexis: Thanks Carter, I'm flattered. But I have a really busy week coming up.\r\nCarter: Yeah, no worries. I totally understand. But if you ever want to go grab dinner again, just let me know. \r\nAlexis: Yeah of course. Thanks again for tonight. \r\nCarter: Sure. Have a great night. "
b'Alexis and Carter met tonight. Carter would like to meet again, but Alexis is busy.'


 We will, therefore, change
the format of the dataset to a dictionary having two keys: `"encoder_text"` and
`"decoder_text"`.

This is how [`keras_hub.models.BartSeq2SeqLMPreprocessor`](https://keras.io/keras_hub/api/models/bart/bart_seq_2_seq_lm_preprocessor/)
expects the input format to be.

In [ ]:
train_ds = (
    samsum_ds.map(
        lambda dialogue, summary: {"encoder_text": dialogue, "decoder_text": summary}
    )
    .batch(BATCH_SIZE)
    .cache()
)
train_ds = train_ds.take(NUM_BATCHES)

## Fine-tune BART

Let's load the model and preprocessor first. We use sequence lengths of 512
and 128 for the encoder and decoder, respectively, instead of 1024 (which is the
default sequence length). This will allow us to run this example quickly
on Colab.

- the preprocessor is attached to the model.
- the preprocessor tokenizes the encoder text
and the decoder text, adds special tokens and pads them. To generate labels
- for auto-regressive training, the preprocessor shifts the decoder text one
position to the right to predict the next token.

https://www.kaggle.com/models/keras/bart

In [ ]:
preprocessor = keras_hub.models.BartSeq2SeqLMPreprocessor.from_preset(
    "bart_base_en",
    encoder_sequence_length=MAX_ENCODER_SEQUENCE_LENGTH,
    decoder_sequence_length=MAX_DECODER_SEQUENCE_LENGTH,
)
bart_lm = keras_hub.models.BartSeq2SeqLM.from_preset(
    "bart_base_en", preprocessor=preprocessor
)

bart_lm.summary()

100%|██████████| 430/430 [00:00<00:00, 792kB/s]


100%|██████████| 649/649 [00:00<00:00, 1.16MB/s]


100%|██████████| 0.99M/0.99M [00:00<00:00, 2.81MB/s]


100%|██████████| 446k/446k [00:00<00:00, 1.80MB/s]


100%|██████████| 532M/532M [00:08<00:00, 62.3MB/s]


...

Model: "bart_seq2_seq_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ decoder_padding_mask          │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoder_token_ids             │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoder_padding_mask          │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoder_token_ids             │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bart_backbone (BartBackbone)  │ [(None, None, 768),       │     139,417,344 │ decoder_padding_mask[0][0… │
│                               │ (None, None, 768)]        │                 │ decoder_token_ids[0][0],   │
│                               │                           │                 │ encoder_padding_mask[0][0… │
│                               │                           │                 │ encoder_token_ids[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50265)       │      38,603,520 │ bart_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 139,417,344 (531.83 MB)

 Trainable params: 139,417,344 (531.83 MB)

 Non-trainable params: 0 (0.00 B)

Define the optimizer and loss. We use the Adam optimizer with a linearly
decaying learning rate. Compile the model.

In [ ]:
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
    epsilon=1e-6,         # added for numerical stability (paper)
    global_clipnorm=1.0,  # gradient clipping.
)

# Exclude layernorm and bias terms from weight decay.
optimizer.exclude_from_weight_decay(var_names=["bias"])
optimizer.exclude_from_weight_decay(var_names=["gamma"])
optimizer.exclude_from_weight_decay(var_names=["beta"])

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

bart_lm.compile(
    optimizer=optimizer,
    loss=loss,
    weighted_metrics=["accuracy"],
)

Let's train the model!

In [ ]:
%%time
bart_lm.fit(train_ds, epochs=EPOCHS)

600/600 ━━━━━━━━━━━━━━━━━━━━ 503s 650ms/step - accuracy: 0.5522 - loss: 0.4331
CPU times: user 2min 53s, sys: 17.2 s, total: 3min 10s
Wall time: 8min 28s


## Generate summaries and evaluate them!

Now that the model has been trained, let's get to the fun part - actually
generating summaries! Let's pick the first 100 samples from the validation set
and generate summaries for them. We will use the default decoding strategy, i.e.,
greedy search.

Generation in KerasHub is highly optimized. It is backed by the power of XLA.
Secondly, key/value tensors in the self-attention layer and cross-attention layer
in the decoder are cached to avoid recomputation at every timestep.

In [ ]:

def generate_text(model, input_text, max_length=200, print_time_taken=False):
    start = time.time()
    output = model.generate(input_text, max_length=max_length)
    end = time.time()
    print(f"Total Time Elapsed: {end - start:.2f}s")
    return output


# Load the dataset.
val_ds = tfds.load("samsum", split="validation", as_supervised=True)
val_ds = val_ds.take(100)

dialogues = []
ground_truth_summaries = []
for dialogue, summary in val_ds:
    dialogues.append(dialogue.numpy())
    ground_truth_summaries.append(summary.numpy())

# Let's make a dummy call - the first call to XLA generally takes a bit longer.
_ = generate_text(bart_lm, "sample text", max_length=MAX_GENERATION_LENGTH)

# Generate summaries.
generated_summaries = generate_text(
    bart_lm,
    val_ds.map(lambda dialogue, _: dialogue).batch(8),
    max_length=MAX_GENERATION_LENGTH,
    print_time_taken=True,
)

Total Time Elapsed: 13.94s
Total Time Elapsed: 44.01s


Let's see some of the summaries.

In [ ]:
len(dialogues)

100

In [ ]:
for dialogue, generated_summary, ground_truth_summary in zip(
    dialogues[3:8], generated_summaries[3:8], ground_truth_summaries[3:8]
):
    print("Dialogue:", dialogue)
    print("Generated Summary:", generated_summary)
    print("Ground Truth Summary:", ground_truth_summary)
    print("=============================")

Dialogue: b"Giny: do we have rice?\r\nRiley: nope, it's finished\r\nGiny: fuck!\r\nGiny: ok, I'll buy"
Generated Summary: Giny and Riley have rice now. Giny will buy it.
Ground Truth Summary: b"Giny and Riley don't have any rice left. Giny will buy some."
Dialogue: b"Jude: i'll be in warsaw at the beginning of december so we could meet again\r\nLeon: !!!\r\nLeon: at the beginning means...?\r\nLeon: cuz I won't be here during the first weekend\r\nJude: 10\r\nJude: but i think it's a monday, so never mind i guess :D\r\nLeon: yeah monday doesn't really work for me :D\r\nLeon: :<\r\nJude: oh well next time :d\r\nLeon: yeah...!"
Generated Summary: Leon won't be in warsaw until Monday.
Ground Truth Summary: b'Jude is coming to Warsaw on the 10th of December and wants to see Leon. Leon has no time.'
Dialogue: b'Hank: Mom were out of ice cream\r\nFay: Is this your way of asking for more?\r\nHank: Mom please!\r\nFay: you have money you can go to the store\r\nHank: yeah but youre going anyway\r\

The generated summaries look awesome! Not bad for a model trained only for 1
epoch and on 5000 examples